In [ ]:
import os
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F

# Get active Snowflake session
session = get_active_session()

# Determine database and schema dynamically
current_context_df = session.sql("SELECT current_database() AS DATABASE_NAME, current_schema() AS SCHEMA_NAME").to_pandas()
database_name = current_context_df.iloc[0, 0]
schema_name = current_context_df.iloc[0, 1]

ENV = schema_name.split('_')[0]  # Extract environment prefix

# Define target schema for raw data
target_schema = f"{ENV}_RAW_SCHEMA"

# Create CSV file format in the target schema
create_file_format_sql = f"""
CREATE OR REPLACE FILE FORMAT {target_schema}.CSV_FORMAT
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    PARSE_HEADER = TRUE
    SKIP_BLANK_LINES = TRUE
    TRIM_SPACE = TRUE
    ENCODING = 'UTF-8'
    NULL_IF = ('.', 'NULL', 'null', '');
"""
session.sql(create_file_format_sql).collect()

# Define NOAA S3 bucket structure
stages = {
    "NOAA_WEATHER_RAW_STAGE": "s3://noaaclimatedata/weatherData/"
}

# Create external stage for NOAA data
for stage, s3_path in stages.items():
    create_stage_sql = f"""
    CREATE OR REPLACE STAGE {target_schema}.{stage}
    STORAGE_INTEGRATION = noaa_s3_integration
    URL = '{s3_path}'
    FILE_FORMAT = {target_schema}.CSV_FORMAT;
    """
    session.sql(create_stage_sql).collect()

# List files in the stage and load them into tables
for stage in stages:
    files = session.sql(f"LIST @{target_schema}.{stage};").collect()
    if not files:
        print("  No files found in stage.")
        continue
    
    for file in files:
        file_path = file[0]  # Full file path in S3
        file_size = file[1]  # File size in bytes
        file_name = os.path.basename(file_path).split(".")[0].upper()
        
        print(f"  {file_path} (Size: {file_size} bytes)")

        # Create table in the target schema
        create_table_sql = f"""
        CREATE OR REPLACE TABLE {database_name}.{target_schema}.{file_name} (
            STATION STRING,
            DATE DATE,
            TEMP FLOAT,
            PRECIP FLOAT,
            WIND_SPEED FLOAT
        );
        """
        session.sql(create_table_sql).collect()
        print(f"✅ Created table: {file_name} in {target_schema}")
        
        # Load data from the S3 stage into the table
        copy_into_sql = f"""
        COPY INTO {database_name}.{target_schema}.{file_name} 
        FROM @{target_schema}.{stage}/{file_name}
        FILE_FORMAT = {target_schema}.CSV_FORMAT
        MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;
        """
        session.sql(copy_into_sql).collect()
        print(f"✅ Data loaded into {target_schema}.{file_name}")
        
        # Update table to replace NULLs (from '.') with 0
        update_sql = f"""
        UPDATE {database_name}.{target_schema}.{file_name}
        SET TEMP = 0 WHERE TEMP IS NULL;
        """
        session.sql(update_sql).collect()
        print(f"✅ Updated {file_name}: replaced NULL with 0 in TEMP column")
